<a href="https://colab.research.google.com/github/AlexyNat/ml/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Разделяем данные</h3>

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.utils import shuffle
import numpy as np

data = pd.read_csv('sample_data/letters.data', header=None)

y = data[0]
data.drop([0], axis=1, inplace=True)
X_train, X_holdout, y_train, y_holdout = train_test_split(data.values, y, test_size=0.3)

<h3>Масштабирование признаков</h3>

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

<h3>Обучение модели MLPClassifier нейронной сети</h3>

In [3]:
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier(random_state=7).fit(X_train_scaled, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


<h3>Проверка точности модели MLPClassifier по тестовой выборке</h3>

In [4]:
beg = 3
end = 15

pred_vals = clf_mlp.predict(X_holdout_scaled[beg:end, :])
print(f'Предсказанные классы: ({end - beg} вариантов)', pred_vals)
print(f'Истинные классы:      ({end - beg} вариантов)', y_holdout.values[beg:end])
print(f'Средняя точность предсказаний: {clf_mlp.score(X_holdout_scaled, y_holdout):.2}')

Предсказанные классы: (12 вариантов) ['R' 'F' 'T' 'B' 'Y' 'H' 'I' 'W' 'G' 'J' 'K' 'K']
Истинные классы:      (12 вариантов) ['R' 'F' 'T' 'B' 'Y' 'H' 'I' 'W' 'G' 'J' 'K' 'X']
Средняя точность предсказаний: 0.94


<h3>Обучение модели Perceptron нейронной сети</h3>

In [5]:
from sklearn.linear_model import Perceptron


clf_perc = Perceptron(max_iter=300, random_state=17).fit(X_train_scaled, y_train)

<h3>Проверка точности модели Perceptron по тестовой выборке</h3>

In [6]:
beg = 3
end = 15

pred_vals = clf_perc.predict(X_holdout_scaled[beg:end, :])

print(f'Предсказанные классы: ({end - beg} вариантов)', pred_vals)
print(f'Истинные классы:      ({end - beg} вариантов)', y_holdout.values[beg:end])
print(f'Средняя точность предсказаний: {clf_perc.score(X_holdout_scaled, y_holdout):.2}')

Предсказанные классы: (12 вариантов) ['R' 'I' 'T' 'B' 'Y' 'P' 'I' 'M' 'H' 'J' 'K' 'X']
Истинные классы:      (12 вариантов) ['R' 'F' 'T' 'B' 'Y' 'H' 'I' 'W' 'G' 'J' 'K' 'X']
Средняя точность предсказаний: 0.5


Параметры модели


hidden_layer_sizestuple, length = n_layers - 2, default=(100,)
Функция активации activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
alpha float, default=0.0001


Регуляризация: penalty {'L2', 'L1', 'elasticnet'}
alpha float, default=0.0001
Критерий остановки: tol float, default=1e-3